<a href="https://colab.research.google.com/github/pulipulichen/colab-File-Browser/blob/main/colab/colab-File-Browser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **docker-web-Apache-Solr**

用Colab快速架搜尋引擎。

1. 按 `Runtime` > `Run all`，或按Ctrl + F9，直接執行腳本即可。
2. 修改 `docker-app/docker-web-Apache-Solr/` 裡的內容即可。

操作說明請看：https://ppt.cc/fqbnwx

# **Scripts**

In [5]:
PROJECT_NAME="docker-web-Apache-Solr"

# 設定配置請看 https://colab.research.google.com/drive/1_aTP6n4rQty6P0Ez4q-OtiaJJIwP3fU5#scrollTo=QEuow7m5Ai6B

測試的方式

1. `Runtime` > `Run all` (Ctrl + F9)
2. 等待Cloudflare網址可連
3. 確認facets裡面 **有** 「國立臺灣大學」
4. 開啟 docker-app/docker-web-Apache-Solr/stopwords.txt
5. 移除「國立臺灣大學」的「#」，移除註解，按Ctrl + S儲存。
6. `Runtime` > `Interrupt execution`
7. `Runtime` > `Run all` (Ctrl + F9)
8. 等待Cloudflare網址可連
9. 確認facets裡面 **沒有** 「國立臺灣大學」

In [6]:
import os

inited = True
if os.path.isfile('.inited') is False:
  inited = False

def curl_local_url(port):
  while True:
    try:
      output = subprocess.check_output(['curl', '-s', 'http://127.0.0.1:' + str(port) + '/']).decode('utf-8')
      print("Success to connect http://127.0.0.1:" + str(port))
      break  # Break out of the loop if successful
    except subprocess.CalledProcessError:
      # print("Check http://127.0.0.1:" + str(port) + "/ failed. Retrying in 5 seconds...")
      time.sleep(5)
      continue  # Retry in case of failure

def wait_for_cloudflare():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.cloudflare.url'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")

def wait_for_docker_web_ready():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.docker-web.ready'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")

  !sleep 10

def keep_waiting():
  while True:
    time.sleep(1)  # Check every 1 second

if inited is False:
  import os
  from subprocess import getoutput
  from google.colab import drive
  import subprocess

def mountGDrive():
  if not os.path.exists("/google-drive"):
    drive.mount('/google-drive')  # access drive
  folder_path = "/google-drive/MyDrive/docker-app/" + PROJECT_NAME + "/"
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  target_folder_path = "/content/docker-app/"
  if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

  command = ["ln", "-s", folder_path, target_folder_path]
  subprocess.run(command, check=True)

if inited is False:
  import os
  import shutil

def check_solrconfig():
    file_path = f'/content/docker-app/{PROJECT_NAME}/solrconfig.xml'
    txt_file_path = f'/content/docker-app/{PROJECT_NAME}/solrconfig.xml.txt'

    if os.path.exists(file_path) and os.path.exists(txt_file_path) is False:
        shutil.copyfile(file_path, txt_file_path)

if inited is False:
  import os
  from subprocess import getoutput
  import subprocess

def clearData():
  stop_udocker()
  command = ["rm", "-rf", "/content/docker-app/" + PROJECT_NAME]
  subprocess.run(command, check=True)

if inited is False:

  !pip install CherryPy==18.8.0
  !pip install udocker==1.3.10
  !udocker --allow-root install
  import subprocess
  import cherrypy
  import time
  from google.colab.output import eval_js
  !rm -rf ./sample_data*

# ==================
if inited is False:
  !pip install pyyaml==6.0.1
  import yaml

def get_config_from_yaml():
  yaml_file_path = f"./.docker-app/{PROJECT_NAME}/docker-build/image/docker-compose-template.yml"

  with open(yaml_file_path, "r") as file:
    yaml_data = yaml.safe_load(file)
  # print(yaml_data)

  # Assuming there is only one service named "app" in your YAML
  webapp_ports = yaml_data.get("services", {}).get("app", {}).get("ports", [])
  WEBAPP_PORT = None

  for port_mapping in webapp_ports:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(port_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_port, container_port = port_mapping.split(":")
        WEBAPP_PORT = host_port
        break

  # ============
  environments = yaml_data.get("services", {}).get("app", {}).get("environment", [])
  LOCAL_VOLUMN_PATH = None
  for env_mapping in environments:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(env_mapping, str) and "LOCAL_VOLUMN_PATH=" in env_mapping:
        # print(port_mapping)
        key, value = env_mapping.split("=")
        LOCAL_VOLUMN_PATH = value
        break

  RUN_COMMAND = None
  for env_mapping in environments:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(env_mapping, str) and "RUN_COMMAND=" in env_mapping:
        # print(port_mapping)
        key, value = env_mapping.split("=")
        RUN_COMMAND = value
        break

  # ============
  app = yaml_data.get("services", {}).get("app", {})
  # print(app)
  IMAGE_NAME = app['image']

  return WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND

# ==================

def stop_udocker():
  #!udocker --allow-root ps
  !udocker --allow-root ps | awk 'NR > 1 {print $1}' | xargs -I {} udocker --allow-root rm {}
  !pgrep java && pkill java && sleep 5

def run_udocker():
  WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()

  # !rm -f /content/docker-app/$PROJECT_NAME/.docker-web.ready || true
  # !rm -f /content/docker-app/$PROJECT_NAME/.cloudflare.url || true
  ![ -e /content/docker-app/$PROJECT_NAME/.cloudflare.url ] && rm /content/docker-app/$PROJECT_NAME/.cloudflare.url
  ![ -e /content/docker-app/$PROJECT_NAME/.docker-web.ready ] && rm /content/docker-app/$PROJECT_NAME/.docker-web.ready

  RUN_IN_BACKGROUND = True
  if WEBAPP_PORT is None:
    RUN_IN_BACKGROUND = False

  if RUN_IN_BACKGROUND is False:
    run_udocker_in_foreground()
  else:
    run_udocker_in_background()

def run_udocker_in_foreground():

  WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()

  # print('Run container in foreground..')
  # subprocess.run(['udocker', '--allow-root', 'run', '-p' ,f'"{WEBAPP_PORT}:{WEBAPP_PORT}"', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME])
  # !udocker --allow-root run -p "$WEBAPP_PORT:$WEBAPP_PORT" --volume="/content/docker-app/$PROJECT_NAME:$LOCAL_VOLUMN_PATH" $IMAGE_NAME

  # print(LOCAL_VOLUMN_PATH)
  # print(RUN_COMMAND)
  !mkdir -p /content/output
  !rm -rf /content/output/*
  !udocker --allow-root run --volume=/content/:$LOCAL_VOLUMN_PATH  --volume=/content/.docker-app/$PROJECT_NAME/app:/app --volume=/content/output:/output $IMAGE_NAME $RUN_COMMAND

  print("Finish")

def run_udocker_in_background():

  if os.path.isfile('.cloudflared') is False:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O .cloudflared
    !chmod a+x .cloudflared

  stop_udocker()

  !mkdir -p /content/docker-app/$PROJECT_NAME

  WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  !rm -rf ./*nohup.out
  !rm -rf ./docker-app/$PROJECT_NAME/.cloudflare.url

  print('Run container in background..')
  # print(['nohup', 'udocker', '--allow-root', 'run', '-p', f'{WEBAPP_PORT}:{WEBAPP_PORT}', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME, '>', '.nohup.out', '2>&1'])
  # subprocess.Popen(['nohup', 'udocker', '--allow-root', 'run', '-p', f'"{WEBAPP_PORT}:{WEBAPP_PORT}"', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME, '>', '.nohup.out', '2>&1'])
  # !udocker --allow-root run -p "$WEBAPP_PORT:$WEBAPP_PORT" --volume="/content/docker-app/$PROJECT_NAME:$LOCAL_VOLUMN_PATH" $IMAGE_NAME &
  # print(['udocker', '--allow-root', 'run', '-p', f'"{WEBAPP_PORT}:{WEBAPP_PORT}"', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME])
  # subprocess.Popen(['udocker', '--allow-root', 'run', '-p', f'{WEBAPP_PORT}:{WEBAPP_PORT}', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME])
  #!rm /content/.docker-app/udocker.sh || true
  ![ -e /content/.docker-app/udocker.sh ] && rm /content/.docker-app/udocker.sh
  !echo "nohup udocker --allow-root run -p $WEBAPP_PORT:$WEBAPP_PORT --volume=/content/docker-app/$PROJECT_NAME:$LOCAL_VOLUMN_PATH $IMAGE_NAME > .nohup.out 2>&1 &" >> /content/.docker-app/udocker.sh
  !chmod +x /content/.docker-app/udocker.sh
  !bash /content/.docker-app/udocker.sh

  cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port' : int(WEBAPP_PORT)})
  # wait_for_cloudflare()
  curl_local_url(WEBAPP_PORT)
  wait_for_docker_web_ready()

  print('============================================================================')
  print('請找到下面訊息的網址。網址末必須為「.trycloudflare.com 」。')
  print('例如：https://organised-norton-talks-autos.trycloudflare.com')
  print('')
  print('如果沒有資料，請中斷執行後，重新執行這個cell。')
  print('============================================================================')
  #!cat /content/docker-app/docker-web-Apache-Solr/solrconfig.xml | grep dc.publisher
  !./.cloudflared --url "http://127.0.0.1:$WEBAPP_PORT"


# ==================

if inited is False:
  %cd /content
  !touch .inited
  inited = True

# ==================

!mkdir -p ./.docker-app

if os.path.isdir('/content/.docker-app/' + PROJECT_NAME) is False:
  %cd /content/.docker-app
  !git clone "https://github.com/pulipulichen/{PROJECT_NAME}.git"
else:
  %cd /content/.docker-app/$PROJECT_NAME
  !git reset --hard
  !git pull --force
%cd /content/

# ==================

def main():
  run_udocker()

/content/.docker-app/docker-web-Apache-Solr
HEAD is now at cf21676 Update README.md
Already up to date.
/content


In [7]:
#clearData()

# **Runtime**

In [ ]:
# mountGDrive() # 開啟Google Drive掛載功能
main()

Info: deleting container: efd0a43c-2e14-3e25-bc6f-a714ee611295
1464
Run container in background..
Success to connect http://127.0.0.1:8983
請找到下面訊息的網址。網址末必須為「.trycloudflare.com 」。
例如：https://organised-norton-talks-autos.trycloudflare.com

如果沒有資料，請中斷執行後，重新執行這個cell。
2025-02-02T10:39:10Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-02-02T10:39:10Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-02-02T10:39:13Z INF +-------------------------------